In [1]:
import pandas as pd
import os
import nltk

In [2]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

In [3]:
LIB = pd.read_csv('data/LIB.csv')
LIB

,title,author,year,genre,cover color,file,txt_str,n_char
0,Hunger Games,Collins,2008,adventure,black,Collins_HungerGames,Books/Collins_HungerGames.txt,542639
1,Catching Fire,Collins,2009,dystopian,red,Collins_CatchingFire,Books/Collins_CatchingFire.txt,649248
2,Mockingjay,Collins,2010,adventure,blue,Collins_Mockingjay,Books/Collins_Mockingjay.txt,561779
3,Divergent,Roth,2011,dystopian,blue,Roth_Divergent,Books/Roth_Divergent.txt,663128
4,Insurgent,Roth,2012,romance,green,Roth_Insurgent,Books/Roth_Insurgent.txt,578407
5,Allegiant,Roth,2013,romance,red,Roth_Allegiant,Books/Roth_Allegiant.txt,603333


In [4]:
os.chdir("/Users/sydneymathiason/Documents/MSDS/Spring/DS5001/final_project")
TOKENS = pd.DataFrame()
for i in range(len(LIB)):
    title = LIB['title'][i].replace(" ", "")
    print(title)
    text_file = LIB['txt_str'][i]
    LINES = pd.DataFrame(open(text_file, 'r').readlines(), columns=['line_str'])
    LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()
    chap_pat = r"^\s*(?:CHAPTER)\s+[A-Z]+"
    chap_lines = LINES.line_str.str.match(chap_pat) # Returns a truth vector
    LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
    LINES.chap_num = LINES.chap_num.ffill()
    LINES = LINES.dropna(subset=['chap_num']) # Remove everything before Chapter 1
    LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
    LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
    CHAPS = LINES.groupby(OHCO[:1])\
        .line_str.apply(lambda x: '\n'.join(x))\
        .to_frame('chap_str')
    CHAPS['chap_str'] = CHAPS.chap_str.str.strip()
    para_pat = r'\n\n+'
    PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
        .to_frame('para_str').sort_index()
    PARAS.index.names = OHCO[:2]
    PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
    PARAS['para_str'] = PARAS['para_str'].str.strip()
    PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
    sent_pat = r'[.?!;:"]+'
    SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
        .to_frame('sent_str')
    SENTS.index.names = OHCO[:3]
    SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
    SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS
    token_pat = r"[\s',-]+"
    TOKENS_new = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
    TOKENS_new.index.names = OHCO[:4]
    TOKENS_new['pos'] = TOKENS_new.pos_tuple.apply(lambda x: x[1])
    TOKENS_new['token_str'] = TOKENS_new.pos_tuple.apply(lambda x: x[0])
    TOKENS_new['term_str'] = TOKENS_new.token_str.str.lower().str.replace(r"\W+", "", regex=True)
    TOKENS_new = TOKENS_new.drop(columns=['pos_tuple'])
    TOKENS_new['pos_group'] = TOKENS_new.pos.str[:2]
    TOKENS_new = TOKENS_new[['token_str', 'term_str', 'pos', 'pos_group']]
    TOKENS_new.to_csv(f"data/TOKENS_{title}.csv")
    TOKENS_new = pd.read_csv(f"data/TOKENS_{title}.csv")
    TOKENS_new['book_num'] = i #add book num column
    TOKENS_new=TOKENS_new.set_index(['book_num']+OHCO)
    TOKENS_new = TOKENS_new.dropna()
    TOKENS = pd.concat([TOKENS, TOKENS_new])

HungerGames
CatchingFire
Mockingjay
Divergent
Insurgent
Allegiant


In [5]:
TOKENS

token_str term_str  pos  \
book_num chap_num para_num sent_num token_num                           
0        1        0        0        0              When     when  WRB   
                                    1                 I        i  PRP   
                                    2              wake     wake  VBP   
                                    3                up       up   RB   
                                    5               the      the   DT   
...                                                 ...      ...  ...   
5        57       122      1        3            mended   mended  VBN   
                           2        0                We       we  PRP   
                                    1              mend     mend  VBP   
                                    2              each     each   DT   
                                    3             other    other   JJ   

                                              pos_group  
book_num chap_num para_num sent_num token_num            
0        1        0        0        0                WR  
                                    1                PR  
                                    2                VB  
                                    3                RB  
                                    5                DT  
...                                                 ...  
5        57       122      1        3                VB  
                           2        0                PR  
                                    1                VB  
                                    2                DT  
                                    3                JJ  

[643782 rows x 4 columns]

In [6]:
TOKENS.to_csv("data/CORPUS.csv")

In [7]:
TOKENS.columns

Index(['token_str', 'term_str', 'pos', 'pos_group'], dtype='object')